##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Neural machine translation with attention

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/nmt_with_attention">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/nmt_with_attention.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This notebook trains a sequence to sequence (seq2seq) model for Spanish to English translation. This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input a Spanish sentence, such as *"¿todavia estan en casa?"*, and return the English translation: *"are you still at home?"*

The translation quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while translating:

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot">

Note: This example takes approximately 10 minutes to run on a single P100 GPU.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [3]:
from tensorflow.python.client import device_lib 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7307376244006776862
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4930941747
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8386544013666488230
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/. This dataset contains language translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

There are a variety of languages available, but we'll use the English-Spanish dataset. For convenience, we've hosted a copy of this dataset on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [4]:
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/wikipedia_tabs.txt"

In [5]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  # w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zçàéêèù .,0-9'\"()\[\]{}=\-;/:«»#%*ÉÈ!?]+", "\x02", w)

  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '\x00' + w + '\x01'
  return w

In [6]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

 may i borrow this book?
b'\x00\x02puedo tomar prestado este libro?\x01'


In [7]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples, max_length):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  lines = [l for l in lines if len(l)<=max_length]
  print("%s lines with max length %s"%(len(lines), max_length))

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [8]:
fr1, fr2 = create_dataset(path_to_file, None, 80)
print(fr1[0])
print(fr2[0])

40650 lines with max length 80
 (p. joanne).
 p. joanne .


In [9]:
def max_length(tensor):
  return max(len(t) for t in tensor)

In [10]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='', char_level=True)
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [11]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  inp_lang, targ_lang = create_dataset(path, num_examples, 80)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [12]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

40650 lines with max length 80


In [13]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [14]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [15]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
7 ---->  
16 ----> c
20 ----> g
2 ----> e
31 ----> 2
1 ---->  
26 ----> f
3 ----> a
45 ----> 6
1 ---->  
45 ----> 6
5 ----> .
8 ----> 

Target Language; index to word mapping
12 ---->  
16 ----> c
20 ----> g
2 ----> e
44 ----> 2
1 ---->  
22 ----> f
6 ----> a
45 ----> 6
1 ---->  
7 ----> s
3 ----> i
23 ----> x
1 ---->  
10 ----> .
13 ----> 


### Create a tf.data dataset

In [16]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

tf.device('/GPU:0')

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 44]), TensorShape([64, 67]))

## Write the encoder and decoder model

Implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence. The below picture and formulas are an example of attention mechanism from [Luong's paper](https://arxiv.org/abs/1508.04025v5). 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

This tutorial uses [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) for the encoder. Let's decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU

The shapes of all the vectors at each step have been specified in the comments in the code:

In [18]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 44, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [20]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [21]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 44, 1)


In [22]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 53)


## Define the optimizer and the loss function

In [24]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [25]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [26]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['\x00']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [27]:
EPOCHS = 120

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.7595
Epoch 1 Batch 100 Loss 0.9615
Epoch 1 Batch 200 Loss 0.8465
Epoch 1 Batch 300 Loss 0.4386
Epoch 1 Loss 0.7584
Time taken for 1 epoch 378.9968886375427 sec

Epoch 2 Batch 0 Loss 0.1708
Epoch 2 Batch 100 Loss 0.1450
Epoch 2 Batch 200 Loss 0.1262
Epoch 2 Batch 300 Loss 0.1062
Epoch 2 Loss 0.1323
Time taken for 1 epoch 328.8066358566284 sec

Epoch 3 Batch 0 Loss 0.0823
Epoch 3 Batch 100 Loss 0.0767
Epoch 3 Batch 200 Loss 0.0435
Epoch 3 Batch 300 Loss 0.0553
Epoch 3 Loss 0.0730
Time taken for 1 epoch 327.1417603492737 sec

Epoch 4 Batch 0 Loss 0.0476
Epoch 4 Batch 100 Loss 0.0635
Epoch 4 Batch 200 Loss 0.0436
Epoch 4 Batch 300 Loss 0.0349
Epoch 4 Loss 0.0528
Time taken for 1 epoch 326.2082414627075 sec

Epoch 5 Batch 0 Loss 0.0448
Epoch 5 Batch 100 Loss 0.0722
Epoch 5 Batch 200 Loss 0.1603
Epoch 5 Batch 300 Loss 0.0970
Epoch 5 Loss 0.0716
Time taken for 1 epoch 326.3894853591919 sec

Epoch 6 Batch 0 Loss 0.1042
Epoch 6 Batch 100 Loss 0.0415
Epoch 6 Batch 200 Loss

Epoch 44 Loss 0.8875
Time taken for 1 epoch 325.23316740989685 sec

Epoch 45 Batch 0 Loss 0.9522
Epoch 45 Batch 100 Loss 0.8048
Epoch 45 Batch 200 Loss 0.8025
Epoch 45 Batch 300 Loss 0.8563
Epoch 45 Loss 0.8317
Time taken for 1 epoch 324.5870668888092 sec

Epoch 46 Batch 0 Loss 0.8803
Epoch 46 Batch 100 Loss 0.7334
Epoch 46 Batch 200 Loss 0.7117
Epoch 46 Batch 300 Loss 0.8107
Epoch 46 Loss 0.7883
Time taken for 1 epoch 325.42580819129944 sec

Epoch 47 Batch 0 Loss 0.8086
Epoch 47 Batch 100 Loss 0.8653
Epoch 47 Batch 200 Loss 0.6881
Epoch 47 Batch 300 Loss 0.7401
Epoch 47 Loss 0.7569
Time taken for 1 epoch 324.8776595592499 sec

Epoch 48 Batch 0 Loss 0.6896
Epoch 48 Batch 100 Loss 0.8408
Epoch 48 Batch 200 Loss 0.6857
Epoch 48 Batch 300 Loss 0.7674
Epoch 48 Loss 0.7186
Time taken for 1 epoch 329.4640769958496 sec

Epoch 49 Batch 0 Loss 0.6263
Epoch 49 Batch 100 Loss 0.6887
Epoch 49 Batch 200 Loss 0.7140
Epoch 49 Batch 300 Loss 0.6762
Epoch 49 Loss 0.6934
Time taken for 1 epoch 324.96849

Epoch 88 Batch 0 Loss 0.4609
Epoch 88 Batch 100 Loss 0.3399
Epoch 88 Batch 200 Loss 0.2332
Epoch 88 Batch 300 Loss 0.2525
Epoch 88 Loss 0.2878
Time taken for 1 epoch 325.7916042804718 sec

Epoch 89 Batch 0 Loss 0.1788
Epoch 89 Batch 100 Loss 0.2646
Epoch 89 Batch 200 Loss 0.1462
Epoch 89 Batch 300 Loss 0.2173
Epoch 89 Loss 0.1919
Time taken for 1 epoch 324.13338017463684 sec

Epoch 90 Batch 0 Loss 0.6245
Epoch 90 Batch 100 Loss 0.8625
Epoch 90 Batch 200 Loss 0.7566
Epoch 90 Batch 300 Loss 0.8574
Epoch 90 Loss 0.8167
Time taken for 1 epoch 326.8307077884674 sec

Epoch 91 Batch 0 Loss 0.6970
Epoch 91 Batch 100 Loss 0.6602
Epoch 91 Batch 200 Loss 0.7011
Epoch 91 Batch 300 Loss 0.5776
Epoch 91 Loss 0.6520
Time taken for 1 epoch 324.4272599220276 sec

Epoch 92 Batch 0 Loss 0.4451
Epoch 92 Batch 100 Loss 0.4050
Epoch 92 Batch 200 Loss 1.0399
Epoch 92 Batch 300 Loss 0.8769
Epoch 92 Loss 0.6836
Time taken for 1 epoch 325.11758613586426 sec

Epoch 93 Batch 0 Loss 0.7937
Epoch 93 Batch 100 Loss 

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [76]:
def evaluate_tensor(inputs):
  attention_plot = np.zeros((max_length_targ, max_length_inp))
    
  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['\x00']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id]

    if targ_lang.index_word[predicted_id] == '\x01':
      return result, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, attention_plot

In [42]:
# input_tensor_val
def evaluate(sentence):

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  return evaluate_tensor(inputs), sentence

In [43]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [50]:
def translate(sentence):
  (result, attention_plot), sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result.encode("latin-1")))

  attention_plot = attention_plot[:len(result), :len(sentence)]
  plot_attention(attention_plot, [c for c in sentence], [c for c in result])

## Restore the latest checkpoint and test

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore('./training_checkpoints\\ckpt-11')
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
from random import randint
i = 33641 # randint(0,len(fr1)-1)

print("correct => ",fr2[i].encode("latin-1"))
translate(fr1[i])


In [ ]:
with open("./test2.target", "w") as fT,\
     open("./test2.predict", "w") as fP,\
     open("./test2.input", "w") as fI:
    for inputs, target in zip(input_tensor_val, target_tensor_val):
        result, _ = evaluate_tensor(tf.expand_dims(inputs,0))
        inputs = "".join([inp_lang.index_word[c] for c in inputs if c != 0])
        target = "".join([targ_lang.index_word[c] for c in target if c != 0])
        fT.write(target+"\n")
        fP.write(result+"\n")
        fI.write(inputs+"\n")
        print(inputs, target, result)
    

 yoshiwara: city of the senses .  yoshiwara city of the senses . yoshiwara city of the senses .
 23 (1971/rév.  vingt-trois dix-neuf cent soixante et onze rév. deux cinze . cingt minze . cingt minze . cingt minze . cingt minze 
 the imperial house of japan.  the imperial house of japan . the imperial house of japan .
 (1834).  dix-huit cent trente-quatre . mille cent quatre .
 sa capitale est pontianak.  sa capitale est pontianak . sa capitale est pontianak .
 1998.  dix-neuf cent quatre-vingt-dix-huit . dix-neuf cent quatre-vingt-dix-neuf cent quatre-vingt-dix-neuf cent
 # renilda soares, épouse de valério.  renilda soares , épouse de valério . renilda soares , épouse de volério .
 markus haller, 2010.  markus haller , deux mille dix . markus haller , deux mille .
 réalisé par les blank et skip gerson.  réalisé par les blank et skip gerson . réalisé par les blank et skip gerson .
 c'était l'opération source, menée le .  cétait lopération source , menée le . 

 * luc brisson.  luc brisson . luc brisson .
 (port.  port . port .
 même cadore se rend spontanément.  même cadore se rend spontanément . même cadore se rend spontanémentanémentanémentanémentanémentanément
 croat., 7, nr.  croat . , sept , n.r. . croat , nr .
 il fut néanmoins rapidement renversé.  il fut néanmoins rapidement renversé. il fut néanmoins rapidement renversé.
 february 2005.  february deux mille cinq . february .
 68, no.  soixante-huit , no . sixant , no .
 part xiii.--noctuidae.  part treize . noctuidae . part t.ini -doctuidae .
 1993.  mille neuf cent quatre-vingt-treize . dix-neuf cent quatre-vingt-tre-vingt-tre-vingt-tre-vingt-tre-vingt-
 notes on formalizing context.  notes on formalising context . notes on formalizentext .
 galilée, 2005.  galilée , deux mille cinq . galilée , deux mille .
 internat.  internat . internat .
 la période de gestation dure un an.  la période de gestation dure un an . la période de gestation dure un 

 ) may attack from either the or the .  may attack from either the or the . may attack from either the .
 * .   
 * banque nationale à hasselt.  banque nationale à hasselt . banque nationale à hasselte .
 *médaille matteucci en 1903.  médaille matteucci en dix-neuf cent trois . médaille matteucci en dix-neuf cent quatre neuf cent quatre neuf ce
 * les fonds de wailly.  les fonds de wailly . les fonds de wailly .
 philadelphe*: ordo lxxix.  philadelpheae ordo soixante-dix-neuf . philadelpheae .
 encore!  encore ! encore !
 gracié par bill clinton.  gracié par bill clinton . gracié par bill clinton .
 14).  un . un .
 in-4.  in moins quatre . in quatre .
 technology: the two-edged sword .  technology the two-edged sword . technology .
 l'ge médian était 43 ans.  lge médian était quarante-trois ans . lge médian était quatrente trant trans .
 * marcel vanhamme, bruxelles.  marcel vanhamme , bruxelles . marcel vanhamme le .
 6 (la maison des enfan

 il a obtenu un diplme en musique.  il a obtenu un diplme en musique . il a obtenu un diplme en musique .
 600604.  six cents six cent quatre . sept .
 12), par.  douze , par . deux , par .
 * coelogyne multiflora schltr.  coelogyne multiflora schltr . coelogyne multiflora schltr .
 penguin 1994.  penguin dix-neuf cent quatre-vingt-quatorze . penguin mille quatre-vingt-quatre-vingt-quatre-vingt-quatre-vingt-q
 lett.  lett . lett .
 6473.  soixante-quatre soixante-treize . sept .
 l'école de milet , éd.  lécole de milet , édition lécole de milet , éd.
 il est alors sous forme humaine.  il est alors sous forme humaine . il est alors sous forme humaine .
 * m.b.b.s.  m.b.b.s. m.b.b.s.
 ) (1996) - lee (t.d.  mille neuf cent quatre-vingt-seize lee t.d. mille neuf .
 oxford university press.  oxford university press . oxford university press .
 ) buch.  buch . buch .
 ii 65).  soixante-cinq . i. .
 (1820)* sciomyzides sveciae.  dix-huit cent vin

 air international , vol.  air international , vol . air international , vol .
 155171.  cent cinquante-cinq cent soixante et onze . cent cent cent cent cent cent cent cent cent cent cent cent cent ce
 la capitale provinciale est isernia.  la capitale provinciale est isernia . la capitale provinciale est isernia .
 1956.  mille neuf cent cinquante-six . dix-neuf cent cinquatre .
 son chef-lieu est rison.  son chef-lieu est rison . son chef-lieu est rison .
 le président actuel est nabih berri.  le président actuel est nabih berri . le président actuel est nabih berri .
 joe and hitsville u.k. .  joe and hitsville u.k. . joe and hitsville u.k. .
 il y composa jusquà la fin de sa vie.  il y composa jusqu' à la fin de sa vie . il y composa jusqu .
 * confrontaciones (1972).  confrontaciones dix-neuf cent soixante-douze . confrontaciones .
 le dé ne se lance pas à la main.  le dé ne se lance pas à la main . le dé ne se lance pas à lance pas à lance pas à lanc

...

 mbi publishing company.  m.b.i. publishing company . mbi publishing company .
 sa capitale est palembang.  sa capitale est palembang . sa capitale est palembang .
 susheela raman enfant d'afrique feat.  susheela raman enfant d.afrique feat . susheela raman enfant d.afrique feat .
 .   
 learning navaho , (vols.  learning navaho , vols . learning navaho , vols .
 en 1927, t.s.  en mille neuf cent vingt-sept , t.s. en mille .
 interstate aviation committee .  interstate aviation committee . interstate aviation committee .
 (1994).  mille neuf cent quatre-vingt-quatorze . mille .
 la mishna est écrite en hébreu.  la mishna est écrite en hébreu . la mishna est écrite est écrite est écrite est écrite est écrite es
 4 dvd's, 16:59:25.  quatre d.v.d.s , seize cinquante-neuf vingt-cinq . quatre .
 ), nueva historia del ecuador , vol.  , nueva historia del ecuador , vol . , nueva historia del .
 son nom béarnais est artics .  son nom béarnais est artics . son 

 ), san pietro (gall.  , san pietro gall . , san pietro .
 les lathrées sont des protocarnivores.  les lathrées sont des protocarnivores . les lathrées sont des protocarnivores .
 moira fut sauvée par les x-men.  moira fut sauvée par les x.-men . moira fut sauvée par les t-men .
 ccxe2 db6 8.  ccxe2 db6 huit . ccen .
 p.p.  p.p. p.p.
 elle a un frère ané, todd sims.  elle a un frère ainé , todd sims . elle a un frère ané , todd somd somd somd somd somd somd somd somd
 2009* jeiteles teutonicus.  deux mille neuf jeiteles teutonicus . deux mille neux mille neux mille neux mille neux mille neux mille n
 mdcciii.  0. m.c.
 explor.  explor . explor .
 issn 0306-5634.  issn trois cent six cinq mille six cent trente-quatre . issn mille .
 notez que le zhuyin a été inclus.  notez que le zhuyin a été inclus . notez que le zhuyin a été inclus .
 time magazine .  time magazine . time magazine .
 kinsey était l'ané de trois enfants.  kinsey était lané de troi

 * gregorino cav.  gregorino cav . gregorino cav .
 # les projets , 2003.  les projets , deux mille trois . les projets , , , , , , , , , , , , , , , , , , , , , , , , , , , ,
 *martino, p. 1993.  martino , p. mille neuf cent quatre-vingt-treize . martino , p. .
 streng.  streng . streng .
 * borton, hugh (1955).  borton , hugh dix-neuf cent cinquante-cinq . borton , hugh .
 ** ruby keeler, actrice canadienne ( ).  ruby keeler , actrice canadienne . ruby keeler , actrice canadien e .
 * wikramanayake, e.d.  wikramanayake , e.d. wikramanayake , e.d.
 , jama.  , jama . , jama .
 (2004).  deux mille quatre . deux mille .
 london.  london . london .
 ::motion adoptée.  motion adoptée. motion adoptée .
 mahieu pour annappes.  mahieu pour annappes . mahieu pour annappes .
 création de selmercanada.  création de selmer canada . création de selmercanada .
 , galilée, 2007.  , galilée , deux mille sept . , galilée , deux mille .
 * a guatiza, le jardi

 le couple divorce en 1927.  le couple divorce en mille neuf cent vingt-sept . le couple divorce en dix-neuf cent .
 anthela est un genre de papillons.  anthela est un genre de papillons . anthela est un genre de papillons .
 gukansh: the future and the past.  gukansh the future and the past . gukanshture and the future and the future and the future and the f
 il ajoute : .  il ajoute . il ajoute .
 il est consacré le .  il est consacré le . il est consacré le .
 87**fantasie en mi mineur, op.  quatre-vingt-sept fantasie en mi mineur , opus quf .
 * le chteau des aubelles ().  le chteau des aubelles . le chteau des aubelles aubelles aubelles aubelles aubelles aubelle
 lockheed p-3 orion.  lockheed p. moins trois orion . lockheed pante .
 de portugal .  de portugal . de portugal .
 ozak gakud zensh .  ozak gakud zensh . ozak gakud , .
 beria en fait partie.  beria en fait partie . beria en fait partie .
 le poivre les assaisonne idéalement.  le poi

 aguya; cat.  aguya , cat . aguya , cat .
 1947.  mille neuf cent quarante-sept . dix-neuf cente-sept cente-sept cente-sept cente-sept cente-sept cen
 cat.).  cat . . cat .
 il divorce en 2004.  il divorce en deux mille quatre . il divorce en deux mille neux mille neux mille neux mille neux mill
 w.s.  w.s. w.s.
 avanti!  avanti ! avanti !
 (2004)* popenker, m. et al.  deux mille quatre popenker , monsieur et al . deux mille neux mille neux mille neux mille neux mille neux mille n
 afcet, paris 1989.  afcet , paris mille neuf cent quatre-vingt-neuf . afcet , paris cent quatre-vingt-neuf cent quatre-vingt-neuf cent qu
 current trends in linguistics (vol.  current trends in linguistics vol . current trends in linguistics trends in linguistics trends in lingu
 * chteau de la dauge, dans le bourg.  chteau de la dauge , dans le bourg . chteau de la dauge , dang .
 ** taxodium rich.  taxodium rich . taxodium .
 deutsch  kufi  arabisch.  deutsch kufi arabisch

In [ ]:
translate(u'¿todavia estan en casa?')

In [ ]:
# wrong translation
translate(u'trata de averiguarlo.')

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
